In [ ]:
%pip install accelerate -U

In [ ]:
!nvidia-smi

In [ ]:
torch.cuda.empty_cache()

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/financial-narrative-summaristion-2022/fns_2020.csv


In [31]:
from tqdm import tqdm
import accelerate
import torch

accelerate.__version__

'0.28.0'

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [4]:
%pip install -U datasets
%pip install -U transformers

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip install rouge --quiet

Note: you may need to restart the kernel to use updated packages.


In [37]:
import datasets
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from torch.utils.data import DataLoader, Dataset
from rouge import Rouge

In [7]:
df = pd.read_csv("/kaggle/input/financial-narrative-summaristion-2022/fns_2020.csv")

In [8]:
def train_validate_test_split(df, train_percent=.8, validate_percent=.1, seed=None):
    np.random.seed(seed)
    perm = np.random.permutation(df.index)
    m = len(df.index)
    train_end = int(train_percent * m)
    validate_end = int(validate_percent * m) + train_end
    train = df.iloc[perm[:train_end]]
    validate = df.iloc[perm[train_end:validate_end]]
    test = df.iloc[perm[validate_end:]]
    return train, validate, test

In [9]:
train, validate, test = train_validate_test_split(df, seed=3)

validate

,Gold Summaries,Annual Reports
462,2 shanks annual report 2008\nfinancial highli...,shanks group plc annual report and accounts 2...
2529,the board remains confident of the\ngroup’s l...,report accounts contents\nintroduction 3\nch...
1692,wwwstatprocom page 1 \nhighlights\nachieved...,annual report accounts 2003\nportfolio analy...
1719,business review yule catto co plc annual re...,yule catto co plc\nannual report 2011\nbeing...
643,annual report and accounts 2012 \nwwwomegadia...,a clear strategy \n focused on delivery\n om...
...,...,...
369,01 persimmon plc annual report 2005 \ndoing w...,persimmon plc annual report december 2005\nwh...
793,financial highlights\nmeggitt’s 2014 results ...,meggitt plc annual report and accounts 201...
926,chairman’s letter \ndear shareholders \nin 20...,riotintocomreportingcentre2013\n2013 annual r...
1815,synchronica plc \nannual report and accounts ...,synchronica plc \nannual report and accounts ...


In [10]:
train_ds = datasets.Dataset.from_dict(train)
validation_ds = datasets.Dataset.from_dict(validate)
test_ds = datasets.Dataset.from_dict(test)
fin_dataset = datasets.DatasetDict({"train":train_ds,"test":test_ds,"validation":validation_ds})

In [11]:
model_name = "google/pegasus-cnn_dailymail"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
#model.resize_token_embeddings(len(tokenizer))
len(tokenizer)

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


96103

In [12]:
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch["Annual Reports"], max_length=1024, truncation=True)

    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch["Gold Summaries"], max_length=128, truncation=True)

    return {
        "input_ids": input_encodings["input_ids"],
        "attention_mask": input_encodings["attention_mask"],
        "labels": target_encodings["input_ids"]
    }

In [13]:
fin_dataset_pt = fin_dataset.map(convert_examples_to_features, batched=True)

Map:   0%|          | 0/2060 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3892: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/259 [00:00<?, ? examples/s]

Map:   0%|          | 0/257 [00:00<?, ? examples/s]

In [14]:
columns = ["input_ids", "labels", "attention_mask"]
fin_dataset_pt.set_format(type="torch", columns=columns)

In [15]:
def data_transform(encode):
    
    l = []
    for i,j,k in zip(encode['input_ids'], encode['attention_mask'], encode['labels']):
        dic = {
            'input_ids': i,
            'attention_mask': j,
            'labels': k
        }
        l.append(dic)
        
    return l

train_encodings = data_transform(fin_dataset_pt["train"])
val_encodings = data_transform(fin_dataset_pt["validation"])
test_encodings = data_transform(fin_dataset_pt["test"])

In [16]:
from transformers import DataCollatorForSeq2Seq, TrainingArguments, Trainer
seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
training_args = TrainingArguments(output_dir="pegasus-fns",
                                 num_train_epochs=1.25,
                                 warmup_steps=500,
                                 per_device_train_batch_size=1,
                                 per_gpu_eval_batch_size=1,
                                 weight_decay=0.01,
                                 logging_steps=10,
                                 push_to_hub=True,
                                 evaluation_strategy='steps',
                                 eval_steps=500,
                                 save_steps=500,
                                 gradient_accumulation_steps=64,
                                 report_to="none")

2024-03-20 19:39:52.318490: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-20 19:39:52.318565: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-20 19:39:52.320081: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [17]:
from huggingface_hub import notebook_login

notebook_login()

In [18]:
trainer = Trainer(model=model,
                 args=training_args,
                 tokenizer=tokenizer,
                 data_collator=seq2seq_data_collator,
                 train_dataset=train_encodings,
                 eval_dataset=val_encodings)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [21]:
fin_dataset_pt["train"]

Dataset({
    features: ['Gold Summaries', 'Annual Reports', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 2060
})

In [19]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss


TrainOutput(global_step=20, training_loss=3.7616479873657225, metrics={'train_runtime': 1387.434, 'train_samples_per_second': 1.856, 'train_steps_per_second': 0.014, 'total_flos': 7389065942777856.0, 'train_loss': 3.7616479873657225, 'epoch': 1.24})

In [29]:
def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]

In [22]:
def calculate_metric_on_test_ds(dataset, metric, model, tokenizer, 
                               batch_size=16, device=device, 
                               column_text="Annual Reports", 
                               column_summary="Gold Summaries"):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text].values.tolist(), batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary].values.tolist(), batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):
        
        inputs = tokenizer(article_batch, max_length=1024,  truncation=True, 
                        padding="max_length", return_tensors="pt")
        
        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device), 
                         length_penalty=0.8, num_beams=8, max_length=128)
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, 
                                clean_up_tokenization_spaces=True) 
               for s in summaries]      
        
        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]
        
        
        metric.add_batch(predictions=decoded_summaries, references=target_batch)
        
    #  Finally compute and return the ROUGE scores.
    score = metric.compute()
    return score

In [25]:
!pip install rouge_score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=f19cb6d3a4ff948e6d6a74fa995227abf2627d397d156bebb6a95690f20c0c4c
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [27]:
rouge_metric = datasets.load_metric('rouge')
rouge_metric

/tmp/ipykernel_270/805187050.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric = datasets.load_metric('rouge')
/opt/conda/lib/python3.10/site-packages/datasets/load.py:756: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


Metric(name: "rouge", features: {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}, usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each prediction
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLSum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/datasets/issues/617
    use_stemmer: Bool indicating whether Porter stemmer should be used to strip word suffixes.
    use_aggregator: Return aggregates if this is set to True
Retu

In [32]:
score = calculate_metric_on_test_ds(
    test, rouge_metric, trainer.model, tokenizer, batch_size = 2, column_text = 'Annual Reports', column_summary= 'Gold Summaries'
)

100%|██████████| 130/130 [08:21<00:00,  3.86s/it]


In [33]:
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )

pd.DataFrame(rouge_dict, index = [f'pegasus'] )

,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.037392,0.003115,0.030531,0.0371


In [34]:
model.save_pretrained("model")
tokenizer.save_pretrained("tokenizer")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8, 'forced_eos_token_id': 1}


('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/spiece.model',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

In [35]:
tokenizer = AutoTokenizer.from_pretrained("/kaggle/working/tokenizer")
model = "/kaggle/working/model"

In [38]:
gen_kwargs = {"length_penalty": 0.8, "num_beams":8, "max_length": 128}

sumarizar = pipeline("summarization", model=model,tokenizer=tokenizer)

In [42]:
test.head()

,Gold Summaries,Annual Reports
1178,06\nrenew holdings plc\nannual report and acc...,renew holdings plc annual report and accounts...
1215,05 ashtead group plc annual report and accoun...,ashtead group plc\nannual report and accounts...
797,financial \nhighlights\ntla client angel pag...,2012 annual report contents\nhighlights 2\n...
1298,chief executive’s report\nintroduction\n2011 ...,annual report and accounts 2011 with a nation...
1062,chief executive’s\nstatement\nrapidcloud inte...,rapidcloud international provides cloudbased...


In [45]:
sample_text = test['Annual Reports'][1178][256:512]
orig_summ = test['Gold Summaries'][1178]
predicted_summary = sumarizar(sample_text, **gen_kwargs)[0]["summary_text"]

print("Dialogue:")
print(sample_text)


print("\nReference Summary:")
print(orig_summ)


print("\nModel Summary:")
print(predicted_summary)

Your max_length is set to 128, but your input_length is only 53. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=26)


Dialogue:
	 adjusted 	 operating 	 profit 	
up 78 to £74m 2010 £42m 
  group order book of £285m 2010 £304m including 
engineering services order book up 118 to £179m 
2010 £82m 
  acquisition of engineering services business amco 
now fully integrated
  net debt of

Reference Summary:
 06
renew holdings plc
annual report and accounts 2011 chief executive’s review
renew is now positioned as an engineering 
services group supporting uk infrastructure 
2011 	 has 	 been 	 a 	 significant 	 year 	 for 	 the 	 group 	
which with further organic growth and the 
acquisition of amco has achieved its target 
of shifting the balance of operations into 
engineering services 
engineering services
engineering services revenue was £1767m 
2010 £1274m and now accounts for over 
60 of on‑going group revenue and over 
80 	 of 	 operating 	 profit 	 operating 	 margins 	
improved to 42 2010 33
in engineering services renew targets the 
key markets of energy environmental and 
infrastructure these marke